In [26]:
import pandas as pd

In [5]:
import numpy as np
import re 

df = pd.read_excel('En_In.xls', skiprows = 17, skipfooter = 38, usecols = [2, 3, 4, 5])

df = df.rename(columns={'Unnamed: 2':'Country', 'Petajoules':'Energy Supply', 'Gigajoules':'Energy Supply per Capita', '%':'% Renewable'})

df['Energy Supply'] = pd.to_numeric(df['Energy Supply'], errors='coerce') * 1000000

df['Energy Supply per Capita'] = df['Energy Supply per Capita'].apply(lambda x: np.NaN if x == '...' else x)

df['% Renewable'] = df['% Renewable'].round(3)

pd.options.display.float_format = lambda x: '{:,.3f}'.format(x) if x == df['Energy Supply'].iloc[1] else '{:,.2f}'.format(x)

country_rename = {
    "Republic of Korea": "South Korea",
"United States of America": "United States", 
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"
}

df['Country'] = df['Country'].replace(country_rename)

def remove_parentheses_and_digits(country):
    return re.sub(r'\([^)]*\)|\d+', '', country)

df['Country'] = df['Country'].apply(remove_parentheses_and_digits)

In [79]:
df.loc[df['Country'].isin(['American Samoa', 'South Korea', 'Bolivia '])]

,Country,Energy Supply,Energy Supply per Capita,% Renewable
3,American Samoa,NaN,NaN,0.64
24,Bolivia,"336,000,000.00",32.00,31.48
164,South Korea,"11,007,000,000.00",221.00,2.28


In [6]:
import pandas as pd
gpd_df = pd.read_csv('gpd.csv', skiprows = 4)

country_rename = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong"
}

gpd_df = gpd_df.rename(columns={"Country Name": "Country"})
gpd_df.iloc[0] = gpd_df.iloc[0].replace(country_rename)

gpd_df = gpd_df[gpd_df.columns[gpd_df.columns.astype(str) >= '2006']]

print(gpd_df.head(1))

  Country Country Code                            Indicator Name  \
0   Aruba          ABW  GDP at market prices (constant 2010 US$)   

   Indicator Code  2006  2007  2008  2009             2010  2011  2012  2013  \
0  NY.GDP.MKTP.KD   NaN   NaN   NaN   NaN 2,467,703,910.61   NaN   NaN   NaN   

   2014  2015  
0   NaN   NaN  


In [7]:
import pandas as pd

scimagojr_df = pd.read_excel("scimagojr.xlsx")


print(scimagojr_df.head())

   Rank             Country  Documents  Citable documents  Citations  \
0     1               China     127050             126767     597237   
1     2       United States      96661              94747     792274   
2     3               Japan      30504              30287     223024   
3     4      United Kingdom      20944              20357     206091   
4     5  Russian Federation      18534              18301      34266   

   Self-citations  Citations per document  H index  
0          411683                    4.70      138  
1          265436                    8.20      230  
2           61554                    7.31      134  
3           37874                    9.84      139  
4           12422                    1.85       57  


In [10]:
import pandas as pd

merged_df = pd.merge(scimagojr_df[:15], gpd_df, how='left', left_on='Country', right_on='Country')
merged_df = pd.merge(merged_df, df, how='left', left_on='Country', right_on='Country')

columns_to_keep = ['Country','Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
                   'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable',
                   '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
result_df = merged_df[columns_to_keep].set_index('Country')


print(result_df.head())

                    Rank  Documents  Citable documents  Citations  \
Country                                                             
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   

                    Self-citations  Citations per document  H index  \
Country                                                               
China                       411683                    4.70      138   
United States               265436                    8.20      230   
Japan                        61554                    7.31      134   
United Kingdom               37874                    9.84      139   
Russian Federation           12422                    1.85       57   

                  

In [9]:
result_df.shape

(15, 20)

In [90]:
def task_eight():
    result_df[result_df.columns[9:]] = result_df[result_df.columns[9:]].apply(pd.to_numeric, errors='coerce')
    result_series = result_df[result_df.columns[9:]].mean(axis=1).sort_values(ascending=False)
    return result_series

result = task_eight()
print(result)

Country
United States        15,364,344,302,990.00
China                 5,771,462,666,216.43
Japan                 5,038,370,580,214.73
Germany               3,175,477,580,976.94
United Kingdom        2,487,906,661,418.42
France                2,437,931,487,057.54
Brazil                1,990,721,948,892.61
Italy                 1,927,431,899,942.86
India                 1,608,452,178,732.14
Canada                1,509,679,514,830.64
Russian Federation    1,423,144,980,438.54
Spain                 1,289,162,071,044.99
Australia             1,058,220,663,629.64
South Korea                           2.28
Iran                                   NaN
dtype: float64


In [92]:
def task_nine():
    avg_gdp_series = task_eight()
    country_name = avg_gdp_series.index[5]
    gdp_values = result_df.loc[country_name, '2006':'2015']
    gdp_change = gdp_values['2015'] - gdp_values['2006']
    return country_name, gdp_change

result = task_nine()
print(result)

('France', 153345695364.24023)


In [94]:
def task_ten():
    max_renewable_country = merged_df.loc[merged_df['% Renewable'].idxmax(), 'Country']
    max_renewable_percentage = merged_df['% Renewable'].max()
    return max_renewable_country, max_renewable_percentage

result = task_ten()
print(result)

('Brazil', 69.648)


In [98]:
def task_eleven():
    merged_df['Population Estimate'] = merged_df['Energy Supply'] / merged_df['Energy Supply per Capita']
    merged_df.sort_values(by='Population Estimate', ascending=False, inplace=True)
    
    country_name = merged_df.iloc[5]['Country']
    population = merged_df.iloc[5]['Population Estimate']
    
    return country_name, population

result = task_eleven()
print(result)


('Germany', 80369696.96969697)


In [100]:
def task_twelve():
    merged_df['Citations per Capita'] = merged_df['Citable documents'] / (merged_df['Energy Supply'] / merged_df['Energy Supply per Capita'])
    correlation_value = merged_df['Citations per Capita'].corr(merged_df['Energy Supply per Capita'])
    
    return correlation_value

result = task_twelve()
print(result)


0.8627253903596998


In [102]:
def task_thirteen():
    median_renewable = merged_df['% Renewable'].median()
    merged_df['High Renewable'] = (merged_df['% Renewable'] >= median_renewable).astype(int)
    result_series = merged_df.sort_values(by='Rank')[['Country', 'High Renewable']].set_index('Country')['High Renewable']
    
    return result_series

result = task_thirteen()
print(result)


Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    0
Canada                1
Germany               1
India                 0
France                0
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: High Renewable, dtype: int32
